PostId: 2019-02-14-152312
Title:WhiteBoard Castのエンコーダーをネイティブに変えたいなぁ

WhiteBoardCastで、そろそろ標準のエンコーダーに乗り換えたいな、と思って調べている。

### やりたい事

やりたいのは動画用Canvasと画面表示用Canvasの2つを作って、

1. 動画Canvasは動画に描くものだけをdraw 
2. 画面Canvasはそこにフローティングメニュー等を上書き
3. スクロールする時などのタイミングでCanvasの中身をbitmapとして取り出したり、その次の画面のBitmapを描いたりしたい（前に描いていた所に戻る場合）

くらいの事がやりたい。

アプリの性質として、更新はいつも画面のごく一部、という性質があるので、出来たら毎回全画面送りたくはない。（だが最近は全部送っても平気ならそれでも良い。以前試した時はフレーム落ちが多すぎて全然駄目だったが、今は平気かも？）。

### 提供されているAPI

一方EncoderはByteBufferかSurfaceを介するようになってて、ByteBufferはYUVならとりあえず動きそう。

Canvasを取得出来ればほとんど同じコードが使える。
だか、Surfaceは以下を信じる限りlockSurfaceは動かず、Mから導入されたlockHardwareCanvasを使わないと駄目そう。

[Stackoverflow: Why is video made with MediaCodec garbled for Samsung Galaxy S7?](https://stackoverflow.com/questions/41842726/why-is-video-made-with-mediacodec-garbled-for-samsung-galaxy-s7/41862638#41862638)

自分のメインのスマホはLなので、API Level 22はまだ切れない。

## 考えてる選択肢

とりあえず幾つか書き出しておいて、思いついたら足したりしていく。

### 選択肢1: GLESでbitmapを描く

動画用Canvasにsurfaceを使って、BitmapのdrawなどはGLESのAPIでテクスチャ経由で直接描く事。
これは画面を一部更新する都度全部送る必要はありそうだが、VRAMのフォーマットとかの問題は気にする必要は無い。

ただGLESで画面描くコードを書くのはかったるい。

### 選択肢2: YUVへの変換を自力でやって、ByteBufferを使う

ByteBufferでYUVはだいたい動きそう？そしてByteBufferはオフセットがあるので部分更新が出来そう？（ただニ次元っぽく無いので出来ないかも…その場合でも行ずつ送るのはまぁまぁ早いかもしれない）

ただ自分でYUV変換とかだるくて泣いちゃう。
Bitmap backedなCanvasで、YUVのものが作れればなぁ、と少し調べたが、Bitmapクラスのフラグには無さそうだった。

### 現時点での方針

YUVを手で変換してByteBufferが一番楽そう？

bitmap backedなCanvasはそのままにして、それと同じサイズのYUVバッファを持っておく、変更があれば該当範囲にかかる所だけYUVバッファを更新（偶数番と奇数番の2pixelあれば平気か？）、YUVバッファ自体は全体を送る、くらいでどうだろう？